In [ ]:
import typing as ty              # Python typing hints
import numpy as np               # NumPy numerical library
import numpy.typing as npt       # NumPy type annotations
import matplotlib.pyplot as plt  # Matlab-like plotting library

# Utilty functions:
# Fit a line to (x, y) and return the estimate of y and the slope of the line
#     fit_line(x, y) -> (y_est, a)
# Plot the results returned by `solve`
#     plot_solution(exact, approximate, solver_name) -> a very nice, very pretty plot
from utilities import fit_line, plot_solution

## Introduction

In this exercise, we will be implementing and comparing Euler, predictor-corrector, and Verlet
integration methods to solve the one-dimensional harmonic oscillator

$$
\ddot{x} = -x.
$$

The exact solution to the oscillator is 
$$x(t) = sin(t)$$
$$\dot{x}(t) = v(t) = cos(t),$$
which is provided by the function `exact(t)`.


In [ ]:
def exact(t: npt.ArrayLike) -> tuple[npt.ArrayLike, npt.ArrayLike]:
    """Calculate the exact position and velocity as functions of the time `t`.

    Args:
        t (npt.ArrayLike): Time (N,)

    Returns:
        tuple[npt.ArrayLike, npt.ArrayLike]: Exact position (N,) and velocity (N,)
    """
    x = np.sin(t)  # Exact position x(t) = sin(t)
    v = np.cos(t)  # Exact velocity v(t) = dx(t)/dt = cos(t)
    return x, v

Let's plot the exact solution to see what it looks like.

First, we need a function to help us evaluate the exact and approximate solutions. For that purpose, we've defined the function `solve(x0, v0, N, j, approximate_solver)` below.

In [ ]:
def solve(
    approximate_solver=None,
    x0: float = 0.0,
    v0: float = 1.0,
    N: int = 100,
    j: float = 2.0,
) -> dict[str, ty.Union[None, str, dict[str, npt.ArrayLike]]]:
    """Use `approximate_solver` to compute position and velocity as functions of time with a
    time step 2 * pi * j / N for N steps.
    
    Args:
        x0 (float): Initial position
        v0 (float): Initial velocity
        N (int): Length of the trajectory
        j (float): Number of periods
        approximate_solver: Solver function which takes arguments (x0, v0, dt, N)
    
    Returns:
        dict: Dictionary with keys: 
            'solver_name' (Union[None, str]): name of the approximate solver if provided 
            'exact' (dict[str, npt.ArrayLike]):
                't' (npt.ArrayLike: Time (N,)),
                'x' (npt.ArrayLike: Position (N,)),
                'v' (npt.ArrayLike: Velocity (N,)), and
                'E' (npt.ArrayLike: Energy (N,))
            'approximate' (dict[str, npt.ArrayLike]): same shape as `exact` if approximate
                solver is provided.
    """
    # Compute the time step needed to cover `j` periods in `N` points.
    dt = 2 * np.pi * j / N
    # Generate the vector of times for evaluation
    t = np.arange(N) * dt

    # Evaluate the exact position, velocity, and energy
    x_exact, v_exact = exact(t)
    E_exact = x_exact**2 + v_exact**2
    
    result = {
        'exact': {
            't': t,
            'x': x_exact,
            'v': v_exact,
            'E': E_exact
        },
        'approximate': None,
        'solver_name': None
    }
    
    if approximate_solver is not None:
        # Evaluate the approximate position, velocity, and energy
        x_approx, v_approx = approximate_solver(x0, v0, dt, N)
        E_approx = x_approx**2 + v_approx**2
        result['approximate'] = {
            't': t,
            'x': x_approx,
            'v': v_approx,
            'E': E_approx
        }
        result['solver_name'] = approximate_solver.__name__
    
    return result

Now, let's compute the exact solution with the default parameters and plot it using `plot_solution` from `utilities.py`.

In [ ]:
plot_solution(**solve())
# Note: you can save the plot made by `plot_solution` by providing the keword argument `plot_filename`, e.g.:
#    plot_solution(**solve(), plot_filename="exact.pdf")
# will save the plot to a PDF file.

## Integrators

Two simple integration methods for solving ODEs numerically are Euler and Euler + predictor-corrector methods.

Given the initial conditions $x_0 = x(t_0)$ and $v_0 = v(t_0)$, the solution of $x(t_n)$ and $v(t_n)$ can be computed iteratively by applying the steps of size $dt$.

Euler's method is defined by the steps:
$$x(t_{n + 1}) = x(t_n) + v(t_n) dt,$$
$$v(t_{n + 1}) = v(t_n) - x(t_n) dt.$$

See the function `euler(x0, v0, dt, N)` below for a Python implementation of the algorithm.

In [ ]:
def euler(x0: float, v0: float, dt: float, N: int) -> tuple[npt.ArrayLike, npt.ArrayLike]:
    """Estimate the position and velocity as functions of time `t` using Euler's method
    given an initial positon `x0` and initial velocity `v0`.

    Args:
        x0 (float): Initial position
        v0 (float): Initial velocity
        dt (float): Time step
        N (float): Number of positions and velocities to compute (including the initial
            values)

    Returns:
        tuple[npt.ArrayLike, npt.ArrayLike]: Position and velocity as functions of time.
    """
    # You'll see this trick also in euler_predictor_corrector. We need x[0] = x0 and
    # v[0] = v0. We will overwrite x[1:] and v[1:], below, so we don't care what their
    # values are. So, we just initialize the vectors to be full of their initial value.
    x = np.full(N, x0)  # [x0, x0, ..., x0]
    v = np.full(N, v0)  # [v0, v0, ..., v0]
    
    for i in range(N-1):
        x[i+1] = x[i] + dt * v[i]
        v[i+1] = v[i] - dt * x[i]
    
    return (x, v)

Now we can use Euler's method to solve the ODE and plot its predictions against the exact solution.

In [ ]:
plot_solution(**solve(approximate_solver=euler))

For the first few points, Euler's method is doing alright. However, we see that as the time increases further away from the initial conditions, the approximate solution is drifting away from the exact solution.
One way to improve this is to add the "predictor-corrector" scheme on top of Euler's method.

The predictor-corrector method adds a correction Euler's method:
$$x(t_{n + 1}) = x_\mathrm{Euler}(t_{n + 1}) + \frac{v_\mathrm{Euler}(t_n) + v_\mathrm{Euler}(t_{n + 1})}{2} dt,$$
$$v(t_{n + 1}) = v_\mathrm{Euler}(t_{n + 1}) - \frac{x_\mathrm{Euler}(t_n) + x_\mathrm{Euler}(t_{n + 1})}{2} dt.$$

One can see why this is called "predictor-corrector" because the prediction of velocity/position from Euler's method is used to correct the position/velocity prediction of Euler's method!

A Python implementation is given below in `euler_predictor_corrector(x0, v0, dt, N)`.

In [ ]:
def euler_predictor_corrector(
    x0:float,
    v0: float,
    dt: float,
    N: int
) -> tuple[npt.ArrayLike, npt.ArrayLike]:
    """Estimate the position and velocity as functions of time `t` using the Euler
    predictor-corrector method given an initial positon `x0` and initial velocity `v0`.

    Args:
        x0 (float): Initial position
        v0 (float): Initial velocity
        dt (float): Time step
        N (float): Number of positions and velocities to compute (including the initial
            values)

    Returns:
        tuple[npt.ArrayLike, npt.ArrayLike]: Position and velocity as functions of time.
    """
    x = np.full(N, x0)
    v = np.full(N, v0)
    
    for i in range(N - 1):
        xp = x[i] + dt * v[i]
        vp = v[i] - dt * x[i]
        x[i+1] = x[i] + dt * (v[i] + vp)/2
        v[i+1] = v[i] - dt * (x[i] + xp)/2 
    
    return (x, v)

In [ ]:
plot_solution(**solve(approximate_solver=euler_predictor_corrector))

The "predictor-corrector" adjustment has greatly improved the drift present in the basic Euler's method, great!

## Measuring error
Now that we have two different integrators that we can use to solve the ODE, let's define some error measures to see how well they perform.

Implement the functions `absolute_error`, `drift`, and `rms_error`.

- Absolute error is defined as
    $$\max_t \left| x_\mathrm{calc}(t) - x_\mathrm{exact}(t) \right|$$
- Energy drift is defined as the slope of a linear polynomial fit to $E_\mathrm{calc}(t)$
    $$E_\mathrm{calc}(t) \approx \underbrace{a}_{\mathrm{drift}} t + b$$
- RMS error is defined as
    $$\sqrt{\frac{1}{N-1} \sum_N{\left(E_\mathrm{calc}(t_N) - E_\mathrm{exact}(t_N)\right)^2}}$$

<div class="alert alert-block alert-info"><b>TODO:</b> Complete the functions `absolute_error`, `energy_drift`, and `rms_error`.</div>

In [ ]:
def absolute_error(
    exact: dict[str, npt.ArrayLike],
    approximate: dict[str, npt.ArrayLike],
    **kwargs
) -> float:
    # The inputs `exact` and `approximate` are dictionaries:
    #    {
    #       'x': [...],
    #       'v': [...],
    #       'E': [...],
    #       't': [...] 
    #    }
    # Useful functions: np.max, np.abs
    ## YOUR IMPLEMENTATION ##
    raise NotImplementedError("Implement `absolute_error` and remove this line of code!")
    err = None
    ## YOUR IMPLEMENTATION ##
    return err

In [ ]:
def energy_drift(
    exact: dict[str, npt.ArrayLike],
    approximate: dict[str, npt.ArrayLike],
    **kwargs
) -> float:
    # The inputs `exact` and `approximate` are dictionaries:
    #    {
    #       'x': [...],
    #       'v': [...],
    #       'E': [...],
    #       't': [...] 
    #    }
    # Useful functions: fit_line (in ./utilities.py)
    ## YOUR IMPLEMENTATION ##
    raise NotImplementedError("Implement `energy_drift` and remove this line of code!")
    t = None
    E = None
    E_line, slope = fit_line(t, E)
    drift = None
    ## YOUR IMPLEMENTATION ##
    return drift

In [ ]:
def rms_error(
    exact: dict[str, npt.ArrayLike],
    approximate: dict[str, npt.ArrayLike],
    **kwargs
) -> float:
    # The inputs `exact` and `approximate` are dictionaries:
    #    {
    #       'x': [...],
    #       'v': [...],
    #       'E': [...],
    #       't': [...] 
    #    }
    # Useful functions: np.sqrt, np.sum, np.power, np.size, len, **
    ## YOUR IMPLEMENTATION ##
    raise NotImplementedError("Implement `rms_error` and remove this line of code!")
    err = None
    ## YOUR IMPLEMENTATION ##
    return err

Compute the three error measures with the default parameters to `solve` (except for `approximate_solver`, of course!).
    
- Using Euler integration, the values should be about `1.009, 0.290,` and `0.2`, respectively

<div class="alert alert-block alert-info"><b>TODO:</b> Analyze the errors and the energy drift for Euler and Euler + predictor-corrector integrators.</div>

In [ ]:
solver = euler  # euler, euler_predictor_corrector
result = solve(approximate_solver=solver)

abs_err = absolute_error(**result)
drift = energy_drive(**result)
rms_err = rms_error(**result)

(abs_err, drift, rms_err)

## Improved integrators

We can improve the Euler integrator by adding the second order term to the Taylor expansion of the trajectory $\frac{1}{2}\left(\Delta t\right)^2 f(t)$ (see e.g. equation (3.8) in the "Documentation: integrators in MD" on Moodle).

An alternative integrator is the (velocity-) Verlet method, which was presented in lecture.

Complete the functions below for the 2nd-order Euler method (`euler2nd`), Verlet integration (`verlet`), and optionally for velocity-Verlet (`velocity_verlet`).

<div class="alert alert-block alert-info"><b>TODO:</b> Complete the functions `euler2nd`, `verlet`, and (OPTIONAL) `velocity_verlet`.</div>

In [ ]:
def euler2nd(
    x0: float,
    v0: float,
    dt: float,
    N: int
) -> tuple[npt.ArrayLike, npt.ArrayLike]:
    """Estimate the position and velocity as functions of time `t` using the second-order
    Euler's method given an initial positon `x0` and initial velocity `v0`.

    Args:
        x0 (float): Initial position
        v0 (float): Initial velocity
        dt (float): Time step
        N (float): Number of positions and velocities to compute (including the initial
            values)

    Returns:
        tuple[npt.ArrayLike, npt.ArrayLike]: Position and velocity as functions of time.
    """
    ## YOUR IMPLEMENTATION ##
    raise NotImplementedError("Implement `euler2nd` and delete this line of code.")
    x = np.zeros(N)
    v = np.zeros(N)
    # ...
    ## YOUR IMPLEMENTATION ##    
    return (x, v)

In [ ]:
def verlet(
    x0: float,
    v0: float,
    dt: float,
    N: int
) -> tuple[npt.ArrayLike, npt.ArrayLike]:
    """Estimate the position and velocity as functions of time `t` using the Verlet
    integration method given an initial positon `x0` and initial velocity `v0`.

    Args:
        x0 (float): Initial position
        v0 (float): Initial velocity
        dt (float): Time step
        N (float): Number of positions and velocities to compute (including the initial
            values)

    Returns:
        tuple[npt.ArrayLike, npt.ArrayLike]: Position and velocity as functions of time.
    """
    ## YOUR IMPLEMENTATION ##
    raise NotImplementedError("Implement `verlet` and delete this line of code.")
    x = np.zeros(N)
    v = np.zeros(N)
    # ...
    ## YOUR IMPLEMENTATION ##
    return (x, v)

In [ ]:
## OPTIONAL
def velocity_verlet(
    x0: float,
    v0: float,
    dt: float,
    N: int
) -> tuple[npt.ArrayLike, npt.ArrayLike]:
    """Estimate the position and velocity as functions of time `t` using the velocity Verlet
    integration method given an initial positon `x0` and initial velocity `v0`.

    Args:
        x0 (float): Initial position
        v0 (float): Initial velocity
        dt (float): Time step
        N (float): Number of positions and velocities to compute (including the initial
            values)

    Returns:
        tuple[npt.ArrayLike, npt.ArrayLike]: Position and velocity as functions of time.
    """
    ## YOUR IMPLEMENTATION ##
    raise NotImplementedError("Implement `velocity_verlet` and delete this line of code.")
    x = np.zeros(N)
    v = np.zeros(N)
    # ...
    ## YOUR IMPLEMENTATION ##
    return (x, v)

## Analyze and compare the integrators

Now we have a small family of integrators and some error measures to evaluate them.

You should
1. Observe the behavior of the integrators which you implemented above (2nd-order Euler, Verlet, optionally velocity-Verlet).
2. Compare the error measures among the different integrators.
3. Plot the error measures for different integrators as a function of $\Delta t$.

You should find that Euler is bad, Euler + predictor-corrector has a very small noise, and Verlet wins for absolute error and energy drift.

<div class="alert alert-block alert-info"><b>TODO:</b> Compare and analyze the behavior of the different integrators (see above).</div>

In [ ]:
## YOUR ANALYSIS ##

## Optional exercises

### Frequency as a function of $\Delta t$

For a longer simulation, find the frequency of the oscillator as a function of $\Delta t$.

In [ ]:
## YOUR SOLUTION ##

### Damping

Now, consider damping in the 1-dimensional harmonic oscillator

$$
\ddot{x} + \gamma\dot{x} + \omega_0^2 x = 0
$$

where $\gamma$ is the damping coefficient, and $\omega_0$ is the undamped angular frequency of the oscillator. Implement damping in your code using the Euler, predictor-corrector, and Verlet algorithms (take the same initial conditions as that of the earlier problem). Vary the ratio $\frac{\gamma}{\omega_0}$ and observe the evolution of the damping behavior.

In [ ]:
## YOUR SOLUTION ##